### STEP 2. GENERATE INTERNAL VALIDATION SETS & MODEL TRAINING####

### RUN ALL THE CODE BLOCK COMBINATIONS BELOW####
### MAKE SURE TO RESTART THE KERNEL BEFORE EACH RUN 

#represents the block number of the code block

## Complete Set 1. positive triples + randomNegative triples (without distanceThreshold) —> run #1, #2, #3
## Complete Set 2. positive triples + restrictedNegative triples (with distanceThreshold) —> run #1, #2, #4


In [ ]:
###FUNCTIONS####

def csvtodict(filename):
    vector_dict = {}
    df_vector_dict = pd.read_csv(filename, header = 0)
    keys = df_vector_dict.iloc[:,0].tolist()
    values = list(range(len(df_vector_dict)))
    for key,value in zip(keys,values):
        vector_dict[key] = value
    return vector_dict



In [2]:
#####1.

import pandas as pd
import os
parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

df_randNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation and Model Development/External Validation Datasets/randomNegatives External Validation Set.csv'))
df_restrictedNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation and Model Development/External Validation Datasets/restrictedNegatives External Validation Set.csv'))
df_diverseNegs = pd.read_csv(os.path.join(parent_dir, 'Step 3 External Validation and Model Development/External Validation Datasets/diverseNegatives External Validation Set.csv'))

unified_columns = [f"{i}" for i in range(604)]
filename = os.path.join(parent_dir, 'Step 1 Data Processing/ROBOKOP+DrugMechDB/ROBOKOP+DrugmechDB Data/ROBOMechDB Processed Triples.csv')
df=pd.read_csv(filename)

df.columns = ['0','1','2']

df_ext = pd.concat([df_randNegs.iloc[:,:3],df_restrictedNegs.iloc[:,:3],df_diverseNegs.iloc[:,:3],df],axis=0)

df_80 = df.sample(n=int(0.8*len(df)), random_state=42)  ##take 80% of these positive triples
df_80_indices = df_80.index.tolist()

pos_neg_triples_dictionary = {}
values = my_list = [1] * len(df_ext)
keys = [df_ext.iloc[i,0] + " " + df_ext.iloc[i,1]+ " " + df_ext.iloc[i,2] for i in range(0,len(df_ext))]
for key,value in zip(keys,values):
    pos_neg_triples_dictionary[key] = value


####Find all unique drugs, diseases, and proteins in this 80% set. 
unique_triples_drug = sorted(list(set(df_80['0'])))
unique_triples_disease =sorted(list(set(df_80['1'])))
unique_triples_protein = sorted(list(set(df_80['2'])))


/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24


In [3]:
####2 Importing created protein, disease, and drug vector dictionaries into here
protein_dict = csvtodict(os.path.join(parent_dir,
                                      'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'))

disease_dict = csvtodict(os.path.join(parent_dir,
                                      'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'))

drug_dict = csvtodict(os.path.join(parent_dir,
                                   'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'))

protein_df = pd.read_csv(os.path.join(parent_dir,
                                      'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Protein Vector Dictionary.csv'),
                         header=0)
disease_df = pd.read_csv(os.path.join(parent_dir,
                                      'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Disease Vector Dictionary.csv'),
                         header=0)
drug_df = pd.read_csv(os.path.join(parent_dir,
                                   'Step 2 Data Embedding/Vector Dictionaries/ROBOMechDB Drug Vector Dictionary.csv'),
                      header=0)

In [ ]:
"""3 Complete Set 1 Development"""

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import random

random_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,42]

model_auc_values = []

positive_triples_vector_array = []
for i in range(0,len(df_80)):
    try:
        drug_vector = drug_df.iloc[drug_dict[df_80.iloc[j,0]],1:201].tolist()
        disease_vector = disease_df.iloc[disease_dict[df_80.iloc[j,1]],1:201].tolist()
        protein_vector = protein_df.iloc[protein_dict[df_80.iloc[j,2]],1:201].tolist()
        
        drug_name = df_80.iloc[j,0]
        disease_name = df_80.iloc[j,1]
        protein_name = df_80.iloc[j,2]
        
        row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
        merged = list(itertools.chain(*row))
        positive_triples_vector_array.append(merged)

    except KeyError:
        continue

positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)

for i in random_seeds:
    random.seed(i)

    negative_triples_array = []
    negative_triples_vector_array = []
    temp = set()
    j=0
    while j < int(1.1*len(positive_triples_dataframe)):  ###create negative triples
        drug = random.sample(unique_triples_drug,k=1)[0]
        protein = random.sample(unique_triples_protein, k=1)[0]
        disease = random.sample(unique_triples_disease,k=1)[0]
        if (drug + " " + disease + " " + protein) in pos_neg_triples_dictionary or (drug + " " + disease + " " + protein) in temp:
            continue
        temp.add(drug + " " + disease + " " + protein)
        negative_triples_array.append([drug, disease, protein])
        j+= 1 

    df_negative_triples = pd.DataFrame(negative_triples_array)
    j=0
    for j in range(0,len(df_negative_triples)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_negative_triples.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_negative_triples.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_negative_triples.iloc[j,2]],1:201].tolist()
        
            drug_name = df_negative_triples.iloc[j,0]
            disease_name = df_negative_triples.iloc[j,1]
            protein_name = df_negative_triples.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[0]]
            merged = list(itertools.chain(*row))
            negative_triples_vector_array.append(merged)

        except KeyError:
            continue

    negative_triples_dataframe = pd.DataFrame(negative_triples_vector_array)
    df = pd.concat([positive_triples_dataframe,negative_triples_dataframe],axis=0,ignore_index=True)
    df.columns = unified_columns
    test = pd.concat([df_diverseNegs, df],axis=0)
    col = ['0', '1', '2']
    print(len(test))
    rem = test.drop_duplicates(subset = col)
    print(len(rem))
    
# Assuming df is already defined and contains your dataset
    shuffled_df = df.sample(frac=1.0, random_state=i) 
    data = shuffled_df.iloc[:, 3:-1].values
    labels = shuffled_df.iloc[:, -1].values
    print(shuffled_df)
    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model
    fold = 1
    auc_values = []

    tf.keras.utils.set_random_seed(i)

    for train_index, test_index in skf.split(data, labels):
        print(f"Fold: {fold}")
    
    # Split the data into training and test sets
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
        tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)), 
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.8),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        
        # Train the model
        model.fit(tf_train_set,
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=40,
                    validation_data = tf_test_set, 
                    callbacks=[early_stopping], 
                    verbose=0)
        
        predictions = model.predict(X_test)
        fpr,tpr, thresholds = roc_curve(y_test,predictions)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        print("AUROC for test data:", roc_auc)
        all_predictions.append(predictions)
        all_test_sets.append(y_test)
        
        ####Before each iteration ends, save each model###
        
        save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Seed {i}.keras'))

        fold += 1
        

    fcv_mean_auc = np.mean(auc_values)
    fcv_std_auc = np.std(auc_values)
    fcv_min_auc = np.min(auc_values)
    fcv_max_auc = np.max(auc_values)
    print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)
    model_auc_values.append([fcv_min_auc,fcv_max_auc,fcv_mean_auc,fcv_std_auc])

fcv_df = pd.DataFrame(model_auc_values, columns=['min_auc', 'max_auc', fcv_mean_auc,fcv_std_auc])

min_auc = np.min(fcv_df.iloc[:,0])
max_auc = np.max(fcv_df.iloc[:,1])
avg_auc = np.mean(fcv_df.iloc[:,2])
std_dev_auc = np.mean(fcv_df.iloc[:,3])

ensemble_stats = np.array([min_auc,max_auc,avg_auc,std_dev_auc])
ensemble_stats = ensemble_stats.reshape(1, -1)
ensemble_stats_df = pd.DataFrame(ensemble_stats, columns = ['min_auc', 'max_auc', 'avg_auc', 'std_dev_auc'])
ensemble_stats_df.to_csv(os.path.join(os.getcwd(), 'Classification Models/Complete Set 1 Models/complete_set_1_ensemble_stats.csv'))


In [ ]:
###4 Complete Set 2 Development

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import auc, roc_curve
import tensorflow as tf
import pandas as pd
import keras
import numpy as np
import itertools
import random

ext_df = pd.read_csv('/Users/eding/Desktop/U24 ROBOKOP Project/External Validation/External Validation Datasets/randomNegatives External Validation Set.csv')

model_auc_values = []

unique_protein_vectors = np.array(protein_df.iloc[:,1:201])
unique_disease_vectors = np.array(disease_df.iloc[:,1:201])
unique_drug_vectors = np.array(drug_df.iloc[:,1:201])

unique_protein_dict_names = np.array(protein_df.iloc[:,0])
unique_disease_dict_names = np.array(disease_df.iloc[:,0])
unique_drug_dict_names = np.array(drug_df.iloc[:,0])


def compute_threshold(vector_dict,n=0.5):
    distances = []
    for i in range((len(vector_dict)-1)):
        smallest_distance = 10000000
        for j in range(i+1, len(vector_dict)):
            distance = np.linalg.norm(vector_dict[i]-vector_dict[j])
            if distance < smallest_distance:
                smallest_distance = distance
        distances.append(smallest_distance)

    distances = np.array(distances)
    print(len(distances))
    print('min', np.min(distances))
    mean_distances = np.sum(distances)/len(distances)
    stdev_distance = np.std(distances)
    print('mean',mean_distances)
    print('stdev',stdev_distance)

    applicability_domain = mean_distances+(n*stdev_distance)

    return applicability_domain

drug_threshold = compute_threshold(unique_drug_vectors)
disease_threshold = compute_threshold(unique_disease_vectors)
protein_threshold = compute_threshold(unique_protein_vectors)

print(drug_threshold)
print(disease_threshold)
print(protein_threshold)

random_seeds = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,42]

for i in random_seeds:
    random.seed(i)
    positive_triples_vector_array = []
    for j in range(0,len(df_80)):
        try:
            drug_vector = drug_df.iloc[drug_dict[df_80.iloc[j,0]],1:201].tolist()
            disease_vector = disease_df.iloc[disease_dict[df_80.iloc[j,1]],1:201].tolist()
            protein_vector = protein_df.iloc[protein_dict[df_80.iloc[j,2]],1:201].tolist()
        
            drug_name = df_80.iloc[j,0]
            disease_name = df_80.iloc[j,1]
            protein_name = df_80.iloc[j,2]
        
            row = [[drug_name],[disease_name],[protein_name],drug_vector,disease_vector,protein_vector,[1]]
            merged = list(itertools.chain(*row))
            positive_triples_vector_array.append(merged)

        except KeyError:
            continue

    positive_triples_dataframe = pd.DataFrame(positive_triples_vector_array)

    random_rows = positive_triples_dataframe.sample(int(len(positive_triples_dataframe)/4), random_state = i)
    

    #we are now going to split these rows into 3 equal chunks. 
    #the triples in the first chunk will have its protein parameter randomized, 
    #the triples in the second chunk will have its disease parameter randomized, and so on

    def generate_negative_triples(data, reference_cols, vector_list, dict_names, threshold, entity_cols, max_count):
        negative_triples = []
        temp = set()
    
        for _, row in data.iterrows():
            reference = np.array(row[reference_cols[0]:reference_cols[1]]).astype(float)
            counter = 0
        
            for k, vector in enumerate(vector_list):
                if np.linalg.norm(reference - vector) < threshold:
                    triple_name = [row[col] if col != 'x' else dict_names[k] for col in entity_cols]
                    new_combo = " ".join(triple_name)
                
                    if new_combo in pos_neg_triples_dictionary or new_combo in temp:
                        continue
                
                    temp.add(new_combo)
                    triple_vector = list(itertools.chain(*[np.array(row[3:reference_cols[0]]).tolist(), vector.tolist() if 'x' in entity_cols else np.array(row[reference_cols[1]:603]).tolist()]))
                    negative_triples.append(list(itertools.chain(*[triple_name, triple_vector, [0]])))
                
                    counter += 1
                    if counter >= max_count:
                        break
    
        return negative_triples

# Split the data
    split_rows = np.array_split(random_rows, 3)
    drug_disease_x, drug_x_protein, x_disease_protein = split_rows

# Generate negative triples for each category
    drug_disease_x_negative = generate_negative_triples(drug_disease_x, (403, 603), unique_protein_vectors, unique_protein_dict_names, protein_threshold, [0, 1, 'x'], 6)
    drug_x_protein_negative = generate_negative_triples(drug_x_protein, (203, 403), unique_disease_vectors, unique_disease_dict_names, disease_threshold, [0, 'x', 2], 6)
    x_disease_protein_negative = generate_negative_triples(x_disease_protein, (3, 203), unique_drug_vectors, unique_drug_dict_names, drug_threshold, ['x', 1, 2], 5)

# Combine all negative triples and create the final dataframe
    negative_df = pd.concat([pd.DataFrame(triples) for triples in [drug_disease_x_negative,    drug_x_protein_negative, x_disease_protein_negative]], ignore_index=True)
    df = pd.concat([positive_triples_dataframe, negative_df], axis=0, ignore_index=True)
    df.columns = unified_columns

    test = pd.concat([df_diverseNegs, df],axis=0)
    col = ['0', '1', '2']
    print(len(test))
    rem = test.drop_duplicates(subset = col)
    print(len(rem))

    shuffled_df = df.sample(frac=1.0, random_state=i)

    data = shuffled_df.iloc[:,3:-1].values
    labels = shuffled_df.iloc[:,-1].values

    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model######
    fold = 1
    auc_values = []

    keras.utils.set_random_seed(i)
    #tf.config.experimental.enable_op_determinism()

    shuffled_df = df.sample(frac=1.0, random_state=i)

    data = shuffled_df.iloc[:,3:-1].values
    labels = shuffled_df.iloc[:,-1].values

    num_folds = 5
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=i)

    all_predictions = []
    all_test_sets = []

    # Perform 5-fold cross-validation to test the accuracy of the model######
    fold = 1
    auc_values = []

    keras.utils.set_random_seed(i)
    #tf.config.experimental.enable_op_determinism()

    for train_index, test_index in skf.split(data, labels):
        print(f"Fold: {fold}")
        print(train_index)
    
        # Split the data into training and test sets
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        tf_train_set = tf.data.Dataset.from_tensor_slices((X_train,y_train))
        tf_test_set = tf.data.Dataset.from_tensor_slices((X_test,y_test))
        
        BATCH_SIZE = 32
        STEPS_PER_EPOCH = len(X_train)//BATCH_SIZE
        tf_train_set = tf_train_set.repeat().batch(BATCH_SIZE)
        tf_test_set = tf_test_set.batch(BATCH_SIZE)

    # Define the learning rate schedule
        lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
            0.001,
            decay_steps=STEPS_PER_EPOCH * 1000,
            decay_rate=1,
            staircase=False
        )
        
        def get_optimizer():
            return tf.keras.optimizers.Adam(lr_schedule)

        optimizer = get_optimizer()
        
        model = tf.keras.models.Sequential([
            tf.keras.Input(shape=(600,)),
            tf.keras.layers.Dense(1024, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)), tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

        # Implement early stopping
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        # Train the model
        model.fit(tf_train_set,
                        steps_per_epoch=STEPS_PER_EPOCH, 
                        epochs=100,
                        validation_data = tf_test_set, 
                        callbacks=[early_stopping], 
                        verbose=0)
        
        predictions = model.predict(X_test)
        fpr,tpr, thresholds = roc_curve(y_test,predictions)
        roc_auc = auc(fpr, tpr)
        auc_values.append(roc_auc)
        print("AUROC for test data:", roc_auc)
        all_predictions.append(predictions)
        all_test_sets.append(y_test)
        
        ####Before each iteration ends, save each model###
        
        save_model= tf.keras.models.save_model(model, os.path.join(os.getcwd(), f'Classification Models/Complete Set 2 Models/ROBOMechDB Complete Set 2 Model Seed {i}.keras'))

        fold += 1

    fcv_mean_auc = np.mean(auc_values)
    fcv_std_auc = np.std(auc_values)
    fcv_min_auc = np.min(auc_values)
    fcv_max_auc = np.max(auc_values)
    print(fcv_mean_auc, fcv_min_auc,fcv_max_auc)
    model_auc_values.append([fcv_min_auc,fcv_max_auc,fcv_mean_auc,fcv_std_auc])
    
fcv_df = pd.DataFrame(model_auc_values, columns=['min_auc', 'max_auc', fcv_mean_auc,fcv_std_auc])

min_auc = np.min(fcv_df.iloc[:,0])
max_auc = np.max(fcv_df.iloc[:,1])
avg_auc = np.mean(fcv_df.iloc[:,2])
std_dev_auc = np.mean(fcv_df.iloc[:,3])

ensemble_stats = np.array([min_auc,max_auc,avg_auc,std_dev_auc])
ensemble_stats = ensemble_stats.reshape(1, -1)
ensemble_stats_df = pd.DataFrame(ensemble_stats, columns = ['min_auc', 'max_auc', 'avg_auc', 'std_dev_auc'])
ensemble_stats_df.to_csv(os.path.join(os.getcwd(), 'Classification Models/Complete Set 2 Models/complete_set_2_ensemble_stats.csv'))